# 🛡️ Robust 3D GNSS Degradation Model
## Anti-Flickering Architecture with Hysteresis

**Author:** Sofia Buriak  
**Version:** 2.0 (Diploma Final)  
**Date:** January 2026

---

### 🎯 Problem Statement

Previous models exhibited the **"010101" flickering problem** — alert status oscillates rapidly:
```
Safe → Unsafe → Safe → Unsafe (every second)
```

This destroys **Recall** metric because the model misses short attack bursts.

### 💡 Solution Architecture

1. **3D Spatial Error:** $\epsilon_{3D} = \sqrt{hAcc^2 + vAcc^2}$
2. **Soft Target Engineering:** Rolling window smoothing to "spread" attack zones
3. **Stability Features:** `cnoMean_rolling_std`, `vDOP_rolling_std` detect signal instability
4. **Post-Processing Hysteresis:** Eliminates output flickering programmatically

---

In [ ]:
# ============================================================
# CELL 1: IMPORTS & CONFIGURATION
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import (
    mean_absolute_error, 
    mean_squared_error,
    precision_score, 
    recall_score, 
    f1_score,
    classification_report,
    confusion_matrix
)
import warnings
import gc
import os
import json

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# ========== CONFIGURATION ==========
CONFIG = {
    'DATA_PATH': '../data/processed/all_data_compressed.parquet',
    'MODEL_OUTPUT_DIR': '../models/',
    'MODEL_NAME': 'gnss_adaptive_3d',
    
    # Target Engineering
    'SAFE_LIMIT_MM': 5000,      # 5 meters (in mm) — Safe zone
    'FAIL_LIMIT_MM': 50000,     # 50 meters (in mm) — Critical zone
    'SMOOTHING_WINDOW': '5s',   # Anti-flickering: expand attack zone
    
    # Train/Test Split
    'TEST_START_DATE': '2025-12-01',
    
    # Model Hyperparameters (Recall-Optimized)
    'XGB_PARAMS': {
        'n_estimators': 300,
        'max_depth': 8,
        'learning_rate': 0.03,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'objective': 'reg:logistic',  # Output 0..1
        'tree_method': 'hist',
        'n_jobs': -1,
        'random_state': 42
    },
    
    # Hysteresis Thresholds
    'HYSTERESIS_HIGH': 0.8,  # Enable alarm when score > 0.8
    'HYSTERESIS_LOW': 0.4,   # Disable alarm ONLY when score < 0.4
    
    # Feature Correlation Threshold
    'MIN_CORRELATION': 0.05
}

print("✅ Configuration loaded successfully")
print(f"   Smoothing Window: {CONFIG['SMOOTHING_WINDOW']}")
print(f"   Hysteresis Band: [{CONFIG['HYSTERESIS_LOW']}, {CONFIG['HYSTERESIS_HIGH']}]")

---
## 📊 Step 1: Data Loading & Initial Exploration
---

In [ ]:
# ============================================================
# CELL 2: DATA LOADING
# ============================================================
print("📂 Loading GNSS data...")

# Try parquet first, fall back to CSV loading
try:
    df = pd.read_parquet(CONFIG['DATA_PATH'])
    print(f"   ✅ Loaded from parquet: {df.shape}")
except FileNotFoundError:
    print("   ⚠️ Parquet not found. Loading from raw CSV files...")
    from src.data_loader import load_all_data
    df = load_all_data('../data/raw')

# Sort by time (critical for time-series!)
df = df.sort_values('timestamp').reset_index(drop=True)

# Ensure datetime type
if not np.issubdtype(df['timestamp'].dtype, np.datetime64):
    df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"\n📈 Dataset Info:")
print(f"   Total samples: {len(df):,}")
print(f"   Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"   Columns: {df.columns.tolist()[:10]}... (total: {len(df.columns)})")

# Quick memory optimization
float_cols = df.select_dtypes(include=['float64']).columns
df[float_cols] = df[float_cols].astype(np.float32)
print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

---
## 🔬 Step 2: Preliminary Data Intelligence (Auto-Correlation Analysis)

Before training, we automatically identify which features correlate with:
- **1D Error (vAcc)** — Vertical degradation (altitude)
- **2D Error (hAcc)** — Horizontal degradation (course)
- **3D Error (sqrt(hAcc² + vAcc²))** — Spatial degradation

---

In [ ]:
# ============================================================
# CELL 3: AUTOMATIC CORRELATION ANALYSIS
# ============================================================
print("🔬 PRELIMINARY DATA INTELLIGENCE")
print("="*60)

# Calculate 3D Error
df['error_3d'] = np.sqrt(df['hAcc']**2 + df['vAcc']**2)

# Define potential feature columns (exclude targets and labels)
excluded_cols = ['timestamp', 'hAcc', 'vAcc', 'sAcc', 'tAcc', 'error_3d',
                 'overallPositionLabel', 'horizontalPositionLabel', 
                 'verticalPositionLabel']

feature_candidates = [col for col in df.columns 
                      if col not in excluded_cols 
                      and df[col].dtype in ['float32', 'float64', 'int64', 'int32', 'int8', 'int16']]

print(f"\n📊 Feature candidates: {feature_candidates}\n")

# ========== CORRELATION ANALYSIS ==========
correlation_results = {
    '1D (vAcc)': {},
    '2D (hAcc)': {},
    '3D (Spatial)': {}
}

for feat in feature_candidates:
    if df[feat].notna().sum() > 100:  # Need enough non-null values
        correlation_results['1D (vAcc)'][feat] = df[feat].corr(df['vAcc'])
        correlation_results['2D (hAcc)'][feat] = df[feat].corr(df['hAcc'])
        correlation_results['3D (Spatial)'][feat] = df[feat].corr(df['error_3d'])

# Convert to DataFrame for visualization
corr_df = pd.DataFrame(correlation_results)
corr_df = corr_df.dropna()

# Sort by 3D correlation (most important)
corr_df = corr_df.reindex(corr_df['3D (Spatial)'].abs().sort_values(ascending=False).index)

print("\n📈 CORRELATION TABLE (sorted by 3D impact):")
print("-"*60)
display(corr_df.round(4))

In [ ]:
# ============================================================
# CELL 4: CORRELATION VISUALIZATION
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(16, 6))

targets = ['1D (vAcc)', '2D (hAcc)', '3D (Spatial)']
colors = ['steelblue', 'darkorange', 'forestgreen']

for ax, target, color in zip(axes, targets, colors):
    data = corr_df[target].sort_values()
    bars = ax.barh(data.index, data.values, color=color, alpha=0.7)
    ax.axvline(x=0, color='black', linewidth=0.8)
    ax.axvline(x=CONFIG['MIN_CORRELATION'], color='red', linestyle='--', 
               label=f'Threshold ({CONFIG["MIN_CORRELATION"]})')
    ax.axvline(x=-CONFIG['MIN_CORRELATION'], color='red', linestyle='--')
    ax.set_xlabel('Pearson Correlation')
    ax.set_title(f'{target} Correlations', fontsize=12, fontweight='bold')
    ax.legend(loc='lower right')
    ax.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.suptitle('🔬 Automatic Feature-Error Correlation Analysis', 
             fontsize=14, fontweight='bold', y=1.02)
plt.savefig('../figures/correlation_analysis_3d.png', dpi=150, bbox_inches='tight')
plt.show()

# ========== AUTO-SELECT FEATURES ==========
threshold = CONFIG['MIN_CORRELATION']
auto_selected_features = corr_df[
    (corr_df['3D (Spatial)'].abs() >= threshold) | 
    (corr_df['1D (vAcc)'].abs() >= threshold) |
    (corr_df['2D (hAcc)'].abs() >= threshold)
].index.tolist()

print(f"\n✅ AUTO-SELECTED FEATURES (|corr| >= {threshold}):")
print(f"   {auto_selected_features}")

---
## 🛠️ Step 3: Advanced Feature Engineering

### Physics-Based Features:
1. **Signal Energy:** `cnoMean × numSV` — Total constellation signal power
2. **Satellite Efficiency:** `numSatsTracked / numSV` — Tracking ratio
3. **Geometric Stress:** `vDOP × hDOP` — Combined poor geometry indicator
4. **Temporal Derivatives:** `diff()` and `rolling_std()` for CNO — Signal change rate

### Stability Features (Anti-Flickering):
5. **cnoMean_rolling_std:** High variance = signal instability
6. **vDOP_rolling_std:** High variance = geometry instability

---

In [ ]:
# ============================================================
# CELL 5: ADVANCED FEATURE ENGINEERING
# ============================================================
print("🛠️ ADVANCED FEATURE ENGINEERING")
print("="*60)

# Set timestamp as index for rolling operations
df = df.set_index('timestamp')

# ========== 1. PHYSICS-BASED FEATURES ==========
print("\n📐 Creating physics-based features...")

# 1.1 Signal Energy (Total constellation power)
df['signal_energy'] = (df['cnoMean'] * df['numSV']).astype(np.float32)
print("   ✅ signal_energy = cnoMean × numSV")

# 1.2 Satellite Efficiency (Tracking ratio)
df['sat_efficiency'] = (df['numSV'] / df['numSatsTracked'].replace(0, 1)).clip(0, 5).astype(np.float32)
print("   ✅ sat_efficiency = numSV / numSatsTracked")

# 1.3 Geometric Stress (Combined DOP)
if 'vDOP' in df.columns and 'hDOP' in df.columns:
    df['geometric_stress'] = (df['vDOP'] * df['hDOP']).astype(np.float32)
    print("   ✅ geometric_stress = vDOP × hDOP")
else:
    df['geometric_stress'] = 0.0
    print("   ⚠️ geometric_stress = 0 (vDOP/hDOP not found)")

# 1.4 DOP Ratio (Asymmetry indicator)
if 'vDOP' in df.columns and 'hDOP' in df.columns:
    df['dop_ratio'] = (df['vDOP'] / df['hDOP'].replace(0, 1)).clip(0, 10).astype(np.float32)
    print("   ✅ dop_ratio = vDOP / hDOP")

# ========== 2. TEMPORAL DERIVATIVES ==========
print("\n⏱️ Creating temporal derivatives...")

# 2.1 CNO Rate of Change
df['cnoMean_diff'] = df['cnoMean'].diff().fillna(0).astype(np.float32)
print("   ✅ cnoMean_diff = diff(cnoMean)")

# 2.2 Lag features
for col in ['cnoMean', 'sat_efficiency', 'numSV']:
    if col in df.columns:
        df[f'{col}_lag1'] = df[col].shift(1).bfill().astype(np.float32)
        df[f'{col}_lag3'] = df[col].shift(3).bfill().astype(np.float32)
print("   ✅ Lag features (t-1, t-3) created")

# ========== 3. STABILITY FEATURES (ANTI-FLICKERING) ==========
print("\n📊 Creating stability features (anti-flickering)...")

stability_window = '10s'

# 3.1 CNO stability
df['cnoMean_rolling_mean'] = df['cnoMean'].rolling(stability_window).mean().astype(np.float32)
df['cnoMean_rolling_std'] = df['cnoMean'].rolling(stability_window).std().fillna(0).astype(np.float32)
print(f"   ✅ cnoMean_rolling_std ({stability_window}) — Signal instability indicator")

# 3.2 vDOP stability (if available)
if 'vDOP' in df.columns:
    df['vDOP_rolling_std'] = df['vDOP'].rolling(stability_window).std().fillna(0).astype(np.float32)
    print(f"   ✅ vDOP_rolling_std ({stability_window}) — Geometry instability indicator")

# 3.3 Satellite count stability
df['numSV_rolling_std'] = df['numSV'].rolling(stability_window).std().fillna(0).astype(np.float32)
print(f"   ✅ numSV_rolling_std ({stability_window}) — Constellation instability indicator")

# 3.4 Signal energy rolling features
df['signal_energy_rolling_mean'] = df['signal_energy'].rolling(stability_window).mean().astype(np.float32)

# Reset index
df = df.reset_index()

print("\n✅ Feature engineering complete!")
print(f"   New columns: {len([c for c in df.columns if 'rolling' in c or 'lag' in c or c in ['signal_energy', 'sat_efficiency', 'geometric_stress']])} features added")

---
## 🎯 Step 4: 3D Target Engineering (Soft Target with Anti-Flickering)

### The "010101" Fix:

Instead of using raw `hAcc`, we create a **smoothed soft target**:

1. **3D Error:** $\epsilon_{raw} = \sqrt{hAcc^2 + vAcc^2}$
2. **Smoothing (Key Fix):** Apply `rolling(window=5s, center=True).max()`
   - *Logic:* If there was an attack at time $t$, we "spread" it to neighboring seconds
   - This forces the model to predict attacks earlier (Pre-warning) and hold alerts longer
3. **Score Conversion:** Transform smoothed error to Score 0..1 using logistic curve

---

In [ ]:
# ============================================================
# CELL 6: 3D TARGET ENGINEERING
# ============================================================
print("🎯 3D TARGET ENGINEERING (Anti-Flickering)")
print("="*60)

SAFE = CONFIG['SAFE_LIMIT_MM']
FAIL = CONFIG['FAIL_LIMIT_MM']

# ========== 1. RAW 3D ERROR ==========
df['error_3d_raw'] = np.sqrt(df['hAcc']**2 + df['vAcc']**2).astype(np.float32)
print(f"   ✅ Raw 3D Error: sqrt(hAcc² + vAcc²)")
print(f"      Range: [{df['error_3d_raw'].min():.0f}, {df['error_3d_raw'].max():.0f}] mm")

# ========== 2. SMOOTHED ERROR (THE KEY FIX!) ==========
# Using rolling MAX to expand attack zones
df = df.set_index('timestamp')

smoothing_window = CONFIG['SMOOTHING_WINDOW']
df['error_3d_smooth'] = df['error_3d_raw'].rolling(
    window=smoothing_window, 
    center=True,  # Center the window for symmetric expansion
    min_periods=1
).max().astype(np.float32)

df = df.reset_index()
print(f"   ✅ Smoothed 3D Error: rolling({smoothing_window}, center=True).max()")

# ========== 3. DEGRADATION SCORES ==========
# 3.1 Individual dimension scores (for analysis)
df['score_1d'] = ((df['vAcc'] - SAFE) / (FAIL - SAFE)).clip(0, 1).astype(np.float32)  # Altitude
df['score_2d'] = ((df['hAcc'] - SAFE) / (FAIL - SAFE)).clip(0, 1).astype(np.float32)  # Horizontal
df['score_3d_raw'] = ((df['error_3d_raw'] - SAFE) / (FAIL - SAFE)).clip(0, 1).astype(np.float32)

# 3.2 SOFT TARGET (What model will learn)
df['soft_target'] = ((df['error_3d_smooth'] - SAFE) / (FAIL - SAFE)).clip(0, 1).astype(np.float32)

# 3.3 MAX-BASED TARGET (Most conservative)
df['degradation_score'] = df[['score_1d', 'score_2d', 'soft_target']].max(axis=1).astype(np.float32)

print(f"\n📊 TARGET DISTRIBUTION:")
print(f"   Safe (score < 0.1):     {(df['degradation_score'] < 0.1).sum():,} samples")
print(f"   Gray Zone (0.1-0.9):    {((df['degradation_score'] >= 0.1) & (df['degradation_score'] <= 0.9)).sum():,} samples")
print(f"   Critical (score > 0.9): {(df['degradation_score'] > 0.9).sum():,} samples")

In [ ]:
# ============================================================
# CELL 7: VISUALIZE TARGET SMOOTHING EFFECT
# ============================================================
print("📊 Visualizing smoothing effect...")

# Find a region with attack activity
attack_mask = df['degradation_score'] > 0.5
attack_indices = df[attack_mask].index

if len(attack_indices) > 0:
    # Find first significant attack
    center_idx = attack_indices[len(attack_indices)//4]  # 25% into attacks
    start_idx = max(0, center_idx - 200)
    end_idx = min(len(df), center_idx + 200)
    
    subset = df.iloc[start_idx:end_idx].copy()
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Top: Raw vs Smoothed Error
    ax1 = axes[0]
    ax1.plot(subset['timestamp'], subset['error_3d_raw']/1000, 
             color='lightcoral', alpha=0.7, label='Raw 3D Error (spiky)')
    ax1.plot(subset['timestamp'], subset['error_3d_smooth']/1000, 
             color='darkred', linewidth=2, label=f'Smoothed Error ({smoothing_window})')
    ax1.axhline(SAFE/1000, color='green', linestyle='--', label=f'Safe Limit ({SAFE/1000:.0f}m)')
    ax1.axhline(FAIL/1000, color='red', linestyle='--', label=f'Fail Limit ({FAIL/1000:.0f}m)')
    ax1.set_ylabel('3D Error (meters)')
    ax1.set_title('🔧 Anti-Flickering: Raw vs Smoothed 3D Error', fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, min(subset['error_3d_smooth'].max()/1000 * 1.2, 100))
    
    # Bottom: Score comparison
    ax2 = axes[1]
    ax2.fill_between(subset['timestamp'], 0, subset['score_3d_raw'], 
                     color='gray', alpha=0.3, label='Raw Score (noisy)')
    ax2.plot(subset['timestamp'], subset['soft_target'], 
             color='blue', linewidth=2, label='Soft Target (training)')
    ax2.axhline(0.5, color='orange', linestyle=':', linewidth=2, label='Decision Threshold (0.5)')
    ax2.axhline(CONFIG['HYSTERESIS_HIGH'], color='red', linestyle='--', 
                label=f'Hysteresis High ({CONFIG["HYSTERESIS_HIGH"]})')
    ax2.axhline(CONFIG['HYSTERESIS_LOW'], color='green', linestyle='--', 
                label=f'Hysteresis Low ({CONFIG["HYSTERESIS_LOW"]})')
    ax2.set_ylabel('Degradation Score')
    ax2.set_xlabel('Time')
    ax2.set_title('🎯 Soft Target Covers Raw Spikes (Improves Recall)', fontweight='bold')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1.05)
    
    plt.tight_layout()
    plt.savefig('../figures/target_smoothing_effect.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("⚠️ No attack events found in dataset")

---
## 📋 Step 5: Feature Selection (Final List)
---

In [ ]:
# ============================================================
# CELL 8: FEATURE SELECTION
# ============================================================
print("📋 FEATURE SELECTION")
print("="*60)

# Define forbidden columns (targets and direct accuracy measures)
FORBIDDEN_COLS = [
    'timestamp', 'hAcc', 'vAcc', 'sAcc', 'tAcc',
    'error_3d', 'error_3d_raw', 'error_3d_smooth',
    'score_1d', 'score_2d', 'score_3d_raw', 'soft_target', 'degradation_score',
    'overallPositionLabel', 'horizontalPositionLabel', 'verticalPositionLabel'
]

# Core physics features (mandatory)
CORE_FEATURES = [
    'cnoMean',           # Signal strength
    'cnoStd',            # Signal stability
    'numSV',             # Satellites in view
    'numSatsTracked',    # Satellites tracked
]

# DOP features (geometry)
DOP_FEATURES = ['vDOP', 'hDOP', 'pDOP', 'gDOP']

# Engineered features
ENGINEERED_FEATURES = [
    'sat_efficiency',
    'signal_energy',
    'geometric_stress',
    'dop_ratio',
    'cnoMean_diff'
]

# Stability features (anti-flickering)
STABILITY_FEATURES = [
    'cnoMean_rolling_std',
    'cnoMean_rolling_mean',
    'vDOP_rolling_std',
    'numSV_rolling_std',
    'signal_energy_rolling_mean'
]

# Lag features (temporal context)
LAG_FEATURES = [col for col in df.columns if 'lag' in col]

# Combine all
ALL_CANDIDATE_FEATURES = (CORE_FEATURES + DOP_FEATURES + ENGINEERED_FEATURES + 
                          STABILITY_FEATURES + LAG_FEATURES)

# Filter to only existing columns
FINAL_FEATURES = [f for f in ALL_CANDIDATE_FEATURES 
                  if f in df.columns and f not in FORBIDDEN_COLS]

# Remove duplicates while preserving order
FINAL_FEATURES = list(dict.fromkeys(FINAL_FEATURES))

print(f"\n✅ FINAL FEATURE LIST ({len(FINAL_FEATURES)} features):")
for i, feat in enumerate(FINAL_FEATURES, 1):
    print(f"   {i:2d}. {feat}")

# Fill NaN values
for col in FINAL_FEATURES:
    df[col] = df[col].fillna(0)

---
## ✂️ Step 6: Train/Test Split (Temporal)
---

In [ ]:
# ============================================================
# CELL 9: TRAIN/TEST SPLIT
# ============================================================
print("✂️ TEMPORAL TRAIN/TEST SPLIT")
print("="*60)

split_date = pd.Timestamp(CONFIG['TEST_START_DATE'])

# Create masks
train_mask = df['timestamp'] < split_date
test_mask = df['timestamp'] >= split_date

# Split data
X_train = df.loc[train_mask, FINAL_FEATURES].copy()
y_train = df.loc[train_mask, 'degradation_score'].copy()

X_test = df.loc[test_mask, FINAL_FEATURES].copy()
y_test = df.loc[test_mask, 'degradation_score'].copy()

# Store test timestamps for plotting
test_timestamps = df.loc[test_mask, 'timestamp'].copy()

print(f"\n📊 SPLIT STATISTICS:")
print(f"   Train: {len(X_train):,} samples (before {split_date.date()})")
print(f"   Test:  {len(X_test):,} samples (from {split_date.date()})")
print(f"\n   Train Attack Rate: {(y_train > 0.5).mean()*100:.2f}%")
print(f"   Test Attack Rate:  {(y_test > 0.5).mean()*100:.2f}%")

# Clean up memory
gc.collect()

---
## 🤖 Step 7: Model Training (Recall-Optimized XGBoost)
---

In [ ]:
# ============================================================
# CELL 10: MODEL TRAINING
# ============================================================
print("🤖 TRAINING RECALL-OPTIMIZED XGBOOST")
print("="*60)

# Display hyperparameters
print("\n⚙️ Hyperparameters:")
for key, value in CONFIG['XGB_PARAMS'].items():
    print(f"   {key}: {value}")

# Initialize model
model = XGBRegressor(**CONFIG['XGB_PARAMS'])

# Train
print("\n🏋️ Training...")
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=50
)

print("\n✅ Training complete!")

In [ ]:
# ============================================================
# CELL 11: FEATURE IMPORTANCE
# ============================================================
print("📊 FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Get feature importances
importance_df = pd.DataFrame({
    'feature': FINAL_FEATURES,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=True)

# Plot
plt.figure(figsize=(10, 8))
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(importance_df)))
plt.barh(importance_df['feature'], importance_df['importance'], color=colors)
plt.xlabel('Importance (Gain)')
plt.title('🔬 XGBoost Feature Importance', fontsize=14, fontweight='bold')
plt.grid(True, axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/feature_importance_3d.png', dpi=150, bbox_inches='tight')
plt.show()

# Print top 10
print("\n🏆 TOP 10 FEATURES:")
for i, row in importance_df.tail(10).iloc[::-1].iterrows():
    print(f"   {row['feature']:30s} {row['importance']:.4f}")

---
## 🔄 Step 8: Hysteresis Post-Processing (Anti-Flickering)

### Hysteresis Logic:
- **Enable alarm** when Score > 0.8 (high threshold)
- **Disable alarm ONLY** when Score < 0.4 (low threshold)

This creates a "sticky" alert that doesn't flip-flop on small fluctuations.

---

In [ ]:
# ============================================================
# CELL 12: HYSTERESIS FUNCTION
# ============================================================
def apply_hysteresis(predictions, low=0.4, high=0.8):
    """
    Apply hysteresis to predictions to eliminate flickering.
    
    Logic:
    - Alert turns ON when prediction > high
    - Alert turns OFF ONLY when prediction < low
    - Between low and high: maintains previous state
    
    Args:
        predictions: Array of prediction scores (0-1)
        low: Lower threshold (alarm OFF)
        high: Upper threshold (alarm ON)
    
    Returns:
        Binary array (0 = Safe, 1 = Alarm)
    """
    alerts = np.zeros(len(predictions), dtype=np.int8)
    current_state = 0  # Start in Safe state
    
    for i, pred in enumerate(predictions):
        if pred > high:
            current_state = 1  # Turn ON
        elif pred < low:
            current_state = 0  # Turn OFF
        # Else: maintain current state
        alerts[i] = current_state
    
    return alerts

print("✅ Hysteresis function defined")
print(f"   Thresholds: LOW={CONFIG['HYSTERESIS_LOW']}, HIGH={CONFIG['HYSTERESIS_HIGH']}")

In [ ]:
# ============================================================
# CELL 13: PREDICTIONS & POST-PROCESSING
# ============================================================
print("🔮 GENERATING PREDICTIONS")
print("="*60)

# Raw predictions
y_pred_raw = model.predict(X_test)

# Apply hysteresis
y_pred_hysteresis = apply_hysteresis(
    y_pred_raw, 
    low=CONFIG['HYSTERESIS_LOW'], 
    high=CONFIG['HYSTERESIS_HIGH']
)

# Simple threshold for comparison
y_pred_simple = (y_pred_raw > 0.5).astype(int)

# Ground truth binary labels
y_true_binary = (y_test > 0.5).astype(int)

print(f"\n📊 Prediction Distribution (Raw):")
print(f"   Min: {y_pred_raw.min():.4f}")
print(f"   Max: {y_pred_raw.max():.4f}")
print(f"   Mean: {y_pred_raw.mean():.4f}")

---
## 📈 Step 9: Multi-Dimensional Validation
---

In [ ]:
# ============================================================
# CELL 14: REGRESSION METRICS
# ============================================================
print("📈 MULTI-DIMENSIONAL VALIDATION")
print("="*60)

# Regression metrics
mae = mean_absolute_error(y_test, y_pred_raw)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_raw))

print(f"\n🎯 REGRESSION METRICS (Score 0-1):")
print(f"   MAE (Mean Absolute Error):  {mae:.4f}")
print(f"   RMSE (Root Mean Squared):   {rmse:.4f}")

# Dimensional analysis
print(f"\n📐 DIMENSIONAL ANALYSIS:")
test_df_metrics = df[test_mask].copy()
test_df_metrics['prediction'] = y_pred_raw

# 1D (Altitude) - using vAcc correlation
mae_1d = mean_absolute_error(test_df_metrics['score_1d'], y_pred_raw)
print(f"   1D (Altitude/vAcc):  MAE = {mae_1d:.4f}")

# 2D (Horizontal) - using hAcc correlation  
mae_2d = mean_absolute_error(test_df_metrics['score_2d'], y_pred_raw)
print(f"   2D (Horizontal/hAcc): MAE = {mae_2d:.4f}")

# 3D (Combined)
mae_3d = mean_absolute_error(test_df_metrics['score_3d_raw'], y_pred_raw)
print(f"   3D (Spatial/Combined): MAE = {mae_3d:.4f}")

In [ ]:
# ============================================================
# CELL 15: CLASSIFICATION METRICS (RECALL FOCUS)
# ============================================================
print("\n🎯 CLASSIFICATION METRICS (Attack Detection)")
print("="*60)

# Simple threshold metrics
print("\n📊 SIMPLE THRESHOLD (0.5):")
print(classification_report(y_true_binary, y_pred_simple, 
                           target_names=['Safe', 'Attack']))

recall_simple = recall_score(y_true_binary, y_pred_simple)
precision_simple = precision_score(y_true_binary, y_pred_simple, zero_division=0)
f1_simple = f1_score(y_true_binary, y_pred_simple)

# Hysteresis metrics
print("\n📊 WITH HYSTERESIS (Anti-Flickering):")
print(classification_report(y_true_binary, y_pred_hysteresis, 
                           target_names=['Safe', 'Attack']))

recall_hysteresis = recall_score(y_true_binary, y_pred_hysteresis)
precision_hysteresis = precision_score(y_true_binary, y_pred_hysteresis, zero_division=0)
f1_hysteresis = f1_score(y_true_binary, y_pred_hysteresis)

# Comparison
print("\n" + "="*60)
print("🏆 RECALL COMPARISON (Attack Class):")
print(f"   Simple Threshold:  Recall = {recall_simple:.4f}")
print(f"   With Hysteresis:   Recall = {recall_hysteresis:.4f}")
print(f"   Improvement:       {(recall_hysteresis - recall_simple)*100:+.2f}%")

In [ ]:
# ============================================================
# CELL 16: FLICKERING ANALYSIS
# ============================================================
print("\n⚡ FLICKERING ANALYSIS")
print("="*60)

# Count state transitions
transitions_simple = np.abs(np.diff(y_pred_simple)).sum()
transitions_hysteresis = np.abs(np.diff(y_pred_hysteresis)).sum()

print(f"\n📊 STATE TRANSITIONS (flickering indicator):")
print(f"   Simple Threshold:  {transitions_simple:,} transitions")
print(f"   With Hysteresis:   {transitions_hysteresis:,} transitions")
print(f"   Reduction:         {(1 - transitions_hysteresis/transitions_simple)*100:.1f}% fewer transitions")

In [ ]:
# ============================================================
# CELL 17: CONFUSION MATRICES
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Simple threshold
cm_simple = confusion_matrix(y_true_binary, y_pred_simple)
sns.heatmap(cm_simple, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Safe', 'Attack'], yticklabels=['Safe', 'Attack'])
axes[0].set_title(f'Simple Threshold (Recall={recall_simple:.3f})', fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Hysteresis
cm_hysteresis = confusion_matrix(y_true_binary, y_pred_hysteresis)
sns.heatmap(cm_hysteresis, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['Safe', 'Attack'], yticklabels=['Safe', 'Attack'])
axes[1].set_title(f'With Hysteresis (Recall={recall_hysteresis:.3f})', fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.suptitle('🎯 Confusion Matrix Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/confusion_matrix_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 📊 Step 10: Attack Fragment Visualization
---

In [ ]:
# ============================================================
# CELL 18: ATTACK FRAGMENT VISUALIZATION
# ============================================================
print("📊 VISUALIZING ATTACK FRAGMENT")
print("="*60)

# Create visualization dataframe
viz_df = pd.DataFrame({
    'timestamp': test_timestamps.values,
    'actual_score': y_test.values,
    'predicted_raw': y_pred_raw,
    'alert_simple': y_pred_simple,
    'alert_hysteresis': y_pred_hysteresis,
    'actual_binary': y_true_binary.values
})

# Find the most "chaotic" region (most flickering in simple predictions)
viz_df['flicker'] = viz_df['alert_simple'].diff().abs()
viz_df['flicker_rolling'] = viz_df['flicker'].rolling(60).sum()
chaos_idx = viz_df['flicker_rolling'].idxmax()

if pd.notna(chaos_idx):
    chaos_time = viz_df.loc[chaos_idx, 'timestamp']
    start_time = chaos_time - pd.Timedelta(seconds=120)
    end_time = chaos_time + pd.Timedelta(seconds=120)
    
    subset = viz_df[(viz_df['timestamp'] >= start_time) & 
                    (viz_df['timestamp'] <= end_time)].copy()
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    
    # Plot 1: Scores
    ax1 = axes[0]
    ax1.plot(subset['timestamp'], subset['actual_score'], 
             color='blue', alpha=0.5, label='Actual (Soft Target)')
    ax1.plot(subset['timestamp'], subset['predicted_raw'], 
             color='red', linewidth=2, label='Predicted Score')
    ax1.axhline(0.5, color='gray', linestyle=':', label='Threshold (0.5)')
    ax1.axhline(CONFIG['HYSTERESIS_HIGH'], color='orange', linestyle='--', alpha=0.7)
    ax1.axhline(CONFIG['HYSTERESIS_LOW'], color='green', linestyle='--', alpha=0.7)
    ax1.fill_between(subset['timestamp'], CONFIG['HYSTERESIS_LOW'], CONFIG['HYSTERESIS_HIGH'],
                     color='yellow', alpha=0.2, label='Hysteresis Band')
    ax1.set_ylabel('Score (0-1)')
    ax1.set_title('🎯 Predicted vs Actual Degradation Score', fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1.05)
    
    # Plot 2: Simple Alert (Flickering)
    ax2 = axes[1]
    ax2.fill_between(subset['timestamp'], 0, subset['actual_binary'],
                     color='blue', alpha=0.3, step='post', label='Actual Attack')
    ax2.step(subset['timestamp'], subset['alert_simple'] + 0.02, 
             where='post', color='red', linewidth=2, label='Simple Alert (FLICKERING)')
    ax2.set_ylabel('Alert State')
    ax2.set_yticks([0, 1])
    ax2.set_yticklabels(['Safe', 'Attack'])
    ax2.set_title('⚠️ Simple Threshold: Notice the "010101" Flickering!', 
                  fontweight='bold', color='red')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Hysteresis Alert (Stable)
    ax3 = axes[2]
    ax3.fill_between(subset['timestamp'], 0, subset['actual_binary'],
                     color='blue', alpha=0.3, step='post', label='Actual Attack')
    ax3.step(subset['timestamp'], subset['alert_hysteresis'] + 0.02, 
             where='post', color='green', linewidth=2, label='Hysteresis Alert (STABLE)')
    ax3.set_ylabel('Alert State')
    ax3.set_yticks([0, 1])
    ax3.set_yticklabels(['Safe', 'Attack'])
    ax3.set_xlabel('Time')
    ax3.set_title('✅ With Hysteresis: Stable Detection, No Flickering', 
                  fontweight='bold', color='green')
    ax3.legend(loc='upper right')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../figures/attack_fragment_hysteresis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Statistics for this fragment
    print(f"\n📊 Fragment Statistics ({len(subset)} samples):")
    print(f"   Simple Alert Transitions:    {subset['alert_simple'].diff().abs().sum():.0f}")
    print(f"   Hysteresis Alert Transitions: {subset['alert_hysteresis'].diff().abs().sum():.0f}")
else:
    print("⚠️ Could not find chaotic region for visualization")

In [ ]:
# ============================================================
# CELL 19: PREDICTION VS REALITY (FULL TEST SET)
# ============================================================
print("📊 PREDICTION VS REALITY (Full Test Set)")
print("="*60)

fig, ax = plt.subplots(figsize=(14, 6))

# Sample for performance (plot every 10th point)
sample_rate = max(1, len(viz_df) // 5000)
plot_df = viz_df.iloc[::sample_rate]

ax.scatter(plot_df['actual_score'], plot_df['predicted_raw'], 
           alpha=0.3, s=5, c='blue')
ax.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Perfect Prediction')

# Add quadrants
ax.axhline(0.5, color='gray', linestyle=':', alpha=0.5)
ax.axvline(0.5, color='gray', linestyle=':', alpha=0.5)

# Labels for quadrants
ax.text(0.25, 0.75, 'False\nAlarm', ha='center', va='center', fontsize=12, color='orange')
ax.text(0.75, 0.25, 'Missed\nAttack', ha='center', va='center', fontsize=12, color='red')
ax.text(0.25, 0.25, 'True\nSafe', ha='center', va='center', fontsize=12, color='green')
ax.text(0.75, 0.75, 'True\nAttack', ha='center', va='center', fontsize=12, color='green')

ax.set_xlabel('Actual Score', fontsize=12)
ax.set_ylabel('Predicted Score', fontsize=12)
ax.set_title('🎯 Prediction vs Reality (Test Set)', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

plt.tight_layout()
plt.savefig('../figures/prediction_vs_reality.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 💾 Step 11: Save Model & Configuration
---

In [ ]:
# ============================================================
# CELL 20: SAVE MODEL
# ============================================================
print("💾 SAVING MODEL & CONFIGURATION")
print("="*60)

# Create output directory
output_dir = CONFIG['MODEL_OUTPUT_DIR']
os.makedirs(output_dir, exist_ok=True)

# Save model
model_path = os.path.join(output_dir, f"{CONFIG['MODEL_NAME']}.json")
model.save_model(model_path)
print(f"   ✅ Model saved to: {model_path}")

# Save configuration
config_to_save = {
    'model_name': CONFIG['MODEL_NAME'],
    'version': '2.0.0',
    'author': 'Sofia Buriak',
    'description': 'Robust 3D GNSS Degradation Model with Anti-Flickering',
    'input_features': FINAL_FEATURES,
    'hyperparameters': CONFIG['XGB_PARAMS'],
    'target_engineering': {
        'safe_limit_mm': CONFIG['SAFE_LIMIT_MM'],
        'fail_limit_mm': CONFIG['FAIL_LIMIT_MM'],
        'smoothing_window': CONFIG['SMOOTHING_WINDOW']
    },
    'hysteresis': {
        'low_threshold': CONFIG['HYSTERESIS_LOW'],
        'high_threshold': CONFIG['HYSTERESIS_HIGH']
    },
    'metrics': {
        'mae': float(mae),
        'rmse': float(rmse),
        'recall_simple': float(recall_simple),
        'recall_hysteresis': float(recall_hysteresis),
        'f1_hysteresis': float(f1_hysteresis)
    }
}

config_path = os.path.join(output_dir, f"{CONFIG['MODEL_NAME']}_config.json")
with open(config_path, 'w') as f:
    json.dump(config_to_save, f, indent=4)
print(f"   ✅ Config saved to: {config_path}")

---
## 📋 Final Summary Report
---

In [ ]:
# ============================================================
# CELL 21: FINAL SUMMARY
# ============================================================
print("\n")
print("="*70)
print("🏆 FINAL SUMMARY REPORT: Robust 3D GNSS Model")
print("="*70)

print(f"""
📊 MODEL ARCHITECTURE:
   • Algorithm:        XGBoost Regressor
   • Features:         {len(FINAL_FEATURES)} physics-based features
   • Target:           3D Degradation Score (0-1)
   • Smoothing:        {CONFIG['SMOOTHING_WINDOW']} window (anti-flickering)

📈 REGRESSION METRICS:
   • MAE:              {mae:.4f}
   • RMSE:             {rmse:.4f}

🎯 CLASSIFICATION METRICS (Attack Detection):
   ┌──────────────────┬────────────┬────────────┐
   │ Method           │ Recall     │ Precision  │
   ├──────────────────┼────────────┼────────────┤
   │ Simple (0.5)     │ {recall_simple:.4f}     │ {precision_simple:.4f}     │
   │ With Hysteresis  │ {recall_hysteresis:.4f}     │ {precision_hysteresis:.4f}     │
   └──────────────────┴────────────┴────────────┘

⚡ FLICKERING ANALYSIS:
   • Simple transitions:    {transitions_simple:,}
   • Hysteresis transitions: {transitions_hysteresis:,}
   • Reduction:             {(1 - transitions_hysteresis/transitions_simple)*100:.1f}%

💾 SAVED ARTIFACTS:
   • Model:  {model_path}
   • Config: {config_path}

🔧 USAGE EXAMPLE:
   >>> import xgboost as xgb
   >>> model = xgb.XGBRegressor()
   >>> model.load_model('{model_path}')
   >>> score = model.predict(X_new)
   >>> alert = apply_hysteresis(score, low=0.4, high=0.8)
""")

print("="*70)
print("✅ MODEL READY FOR DEPLOYMENT!")
print("="*70)